In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Sumarização de textos em documentos grandes


<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/language/examples/reference-architectures/text_summarization_for_large_documents.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/language/examples/reference-architectures/text_summarization_for_large_documents.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/blob/main/language/examples/reference-architectures/text_summarization_for_large_documents.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in Vertex AI Workbench
    </a>
  </td>
</table>

## Visão geral

O resumo de texto é o processo de criar uma versão mais curta de um documento de texto, preservando as informações mais importantes. Isso pode ser útil para várias finalidades, como examinar rapidamente um documento longo, obter a essência de um artigo ou compartilhar um resumo com outras pessoas.

Embora resumir um parágrafo curto seja uma tarefa trivial, há alguns desafios a serem superados se você quiser resumir um documento grande, como um arquivo PDF com várias páginas. Neste notebook, você passará por alguns exemplos de como usar modelos generativos para resumir documentos grandes.

### Objetivo

Neste tutorial, você aprenderá como usar modelos generativos para resumir informações de texto trabalhando com os seguintes exemplos:

- Método de *stuffing*
- Método MapReduce
- MapReduce com Overlapping Chunks
- MapReduce com Rolling Summary

### Custos
Este tutorial usa os seguintes componentes de Google Cloud:

* Vertex AI Studio

Saiba mais sobre possíveis custos envolvidos [preços da Vertex AI](https://cloud.google.com/vertex-ai/pricing),
e use a [Calculadora de preços](https://cloud.google.com/products/calculator/)
para gerar uma estimativa de custo com base no uso projetado.

## Primeiros Passos

### Instalando o SDK da Vertex AI e outras dependências

In [ ]:
!pip install --user google-cloud-aiplatform PyPDF2 ratelimit backoff==1.10.0 --upgrade

**Somente Colab:** Descomente a célula a seguir para reiniciar o kernel ou use o botão para reiniciar o kernel.

In [ ]:
# # Reinicia automaticamente o kernel após as instalações para que seu ambiente possa acessar os novos pacotes
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

### Autenticando seu ambiente de notebook
* Se você estiver usando o **Colab** para executar este notebook, descomente a célula abaixo e continue.
* Se você estiver usando o **Vertex AI Workbench**, confira as instruções de configuração [aqui](../setup-env/README.md).

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

### Importando as bibliotecas necessárias

**Somente Colab:** Descomente a célula a seguir para realizar o processo adequado de inicialização da SDK da Vertex AI.  

In [ ]:
# import vertexai

# PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
# vertexai.init(project=PROJECT_ID, location="us-central1")

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

import warnings
warnings.simplefilter("ignore", UserWarning)

import re
import urllib
import warnings
from pathlib import Path

import backoff
import pandas as pd
import PyPDF2
import ratelimit
from google.api_core import exceptions
from tqdm import tqdm
from vertexai.language_models import TextGenerationModel

warnings.filterwarnings("ignore")

#### Carregando o modelo `text-bison`

In [ ]:
generation_model = TextGenerationModel.from_pretrained("text-bison@001")

### Preparando arquivos de dados

Para começar, você precisará baixar um arquivo pdf para as tarefas de resumo abaixo.

In [ ]:
!pwd
# le o arquivo pdf e cria uma lista de páginas
pdf_file = './documentos/documento.pdf'
reader = PyPDF2.PdfReader(pdf_file)
pages = reader.pages

# lista o conteúdo de 3 páginas do pdf
for i in range(3):
    text = pages[i].extract_text().strip()
    print(f"Page {i}: {text} \n\n")

## Método 1: Stuffing

A maneira mais simples de passar dados para um modelo de linguagem é enviá-los no prompt como contexto. Isso significa simplesmente incluir todas as informações relevantes no prompt, na ordem em que você deseja que o modelo as processe.

Aqui você extrairá o texto de todas as páginas do arquivo pdf.

In [ ]:
# le o pdf e cria a lista de páginas
reader = PyPDF2.PdfReader(pdf_file)
pages = reader.pages

# variável que oncatena todos os textos extraídos
concatenated_text = ""

# loop nas páginas
for page in tqdm(pages):

    # extrai o texto das páginas e remove espaços em branco
    text = page.extract_text().strip()

    # concatena o texto extraído
    concatenated_text += text

print(f"Há {len(concatenated_text)} cadeias de texto no pdf")

Agora você criará um modelo de prompt que pode ser usado posteriormente no notebook.

In [ ]:
prompt_template = """
    Escreva um sumário conciso do texto abaixo delimitado por três aspas invertidas.
    Retorne sua resposta em bullets que cubram os pontos principais do texto.

    ```{text}```

    SUMARIO EM BULLETS:
"""

Aqui você usará o LLM via API para resumir os textos extraídos. Observe que os LLMs atualmente têm limite de texto de entrada e o preenchimento de um texto de entrada grande pode não ser aceito. Você pode ler mais sobre cotas e limites [aqui](https://cloud.google.com/vertex-ai/docs/quotas).

O código a seguir causará **uma exceção (um erro)**!

In [ ]:
# define o prompt usando a template criada anteriormente
prompt = prompt_template.format(text=concatenated_text)

# usa o modelo para sumarizar o texto
summary = generation_model.predict(prompt=prompt, max_output_tokens=1024).text

print(summary)

#### Tentando novamente

O modelo respondeu com uma mensagem de erro: **400 Request contains an invalid argument** porque o texto extraído é muito longo para o modelo generativo processar.

Para evitar esse problema, você inserirá apenas uma parte do texto extraído (por exemplo, as primeiras 30.000 palavras).

In [ ]:
# Define the prompt using the prompt template
prompt = prompt_template.format(text=concatenated_text[:30000])

# Use the model to summarize the text using the prompt
summary = generation_model.predict(prompt=prompt, max_output_tokens=1024).text

print(summary)

### Recapitulando

Embora o texto completo seja muito grande para o modelo, você conseguiu criar uma lista concisa e com marcadores das informações mais importantes de uma parte do PDF usando o modelo. Assim, aqui estão os prós e contras de usar o método de stuffing:

**Prós:**
- Exigiu apenas uma única chamada para o modelo.
- Ao resumir o texto, o modelo tem acesso a todos os dados de uma só vez para que o resultado seja melhor.

**Contras:**
- A maioria dos modelos tem um comprimento de contexto e, para documentos grandes (ou muitos documentos), isso não funcionará, pois resultará em um prompt maior que o comprimento do contexto.
- Este método funciona apenas em pedaços menores de dados e não é adequado para documentos grandes na maioria das vezes.

Na sessão seguinte, você explorará abordagens projetadas para ajudar a lidar com textos mais longos do que o limite de comprimento de contexto dos LLMs.

### Adicionando limite para chamadas de modelo

Ao usar MapReduce ou outros métodos semelhantes, você fará várias chamadas de API para o modelo em um curto período de tempo. Há um limite para o número de chamadas de API que você pode fazer por minuto, portanto, você precisará adicionar uma medida de segurança ao seu código para evitar que o limite seja excedido. Isso ajudará a garantir que seu código seja executado sem problemas e não encontre nenhum erro.

Para este método, aqui estão algumas coisas específicas que você fará:
1. Você usará uma biblioteca Python chamada [ratelimit](https://pypi.org/project/ratelimit/) para limitar o número de chamadas de API por minuto
2. Você usará uma biblioteca Python chamada [backoff](https://pypi.org/project/backoff/) para tentar novamente até que o limite máximo de tempo seja atingido

A função a seguir melhora o processo de chamada de API limitando o número de chamadas a **20 por minuto**. Ele também recua e tenta novamente chamar a API depois de encontrar a exceção **Resource Exhausted**. A duração da espera aumenta **exponencialmente até a marca de 5 minutos**, e então a função desistirá ao tentar novamente.

In [ ]:
CALL_LIMIT = 20  # número de chamadas permitido por intervalo de tempo
ONE_MINUTE = 60  # um minuto em segundos
FIVE_MINUTE = 5 * ONE_MINUTE

# uma função que mostra a mensagem quando acontecer uma nova tentativa
def backoff_hdlr(details):
    print(
        "Backing off {} seconds after {} tries".format(
            details["wait"], details["tries"]
        )
    )


@backoff.on_exception(  # novas tentativas com a estratégia de backoff
    backoff.expo,
    (
        exceptions.ResourceExhausted,
        ratelimit.RateLimitException,
    ),  # exceções para novas tentativas
    max_time=FIVE_MINUTE,
    on_backoff=backoff_hdlr,  # função a ser chamada quando tentando novamente
)
@ratelimit.limits(  # limite do número de chamadas ao modelo por minuto
    calls=CALL_LIMIT, period=ONE_MINUTE
)

# esta função chamará a função `generation_model.predict`, mas tentará novamente se ocorrerem exceções definidas.
def model_with_limit_and_backoff(**kwargs):
    return generation_model.predict(**kwargs)

## Método 2: MapReduce

Esse método funciona primeiro dividindo os dados grandes em blocos e, em seguida, executando um prompt em cada bloco de texto. Para tarefas de resumo, a saída do prompt inicial seria um resumo desse bloco. Uma vez que todas as saídas iniciais tenham sido geradas, um prompt diferente é executado para combiná-los.

Esse método é um pouco mais complexo que o primeiro método, mas pode ser mais eficaz para grandes conjuntos de dados. Aqui você preparará dois modelos de prompt: um para a etapa inicial de resumo e outro para a etapa final de combinação. Você usará esses dois modelos posteriormente neste notebook.

In [ ]:
initial_prompt_template = """
    Escreva um sumário conciso sobre o texto abaixo delimitado por três aspas invertidas.

    ```{text}```

    SUMARIO CONCISO:
"""

final_prompt_template = """
    Escreva um cenário conciso do texto abaixo delimitdo por aspas invertidas.
    Retorne sua resposta em bullets que cubram os pontos principais do texto.

    ```{text}```

    SUMARIO EM BULLETS:
"""

#### Etapa de Map

Nesta seção, você lerá o arquivo PDF novamente e usará o modelo para resumir cada página individualmente usando o modelo de prompt inicial.

In [ ]:
# le o arquivo pdf e cria uma lista de páginas
reader = PyPDF2.PdfReader(pdf_file)
pages = reader.pages

# cria uma lista para armazenar os sumários
initial_summary = []

# iteração nas páginas e eração do sumário de cada página
for page in tqdm(pages):

    # extrai o texto da página e remove os espaços
    text = page.extract_text().strip()

    # cria o prompt usando o texto extraído e a template de prompt
    prompt = initial_prompt_template.format(text=text)

    # gera o sumário usando o modelo e o prompt
    summary = model_with_limit_and_backoff(prompt=prompt, max_output_tokens=1024).text

    # faz o append do sumário da página na lista de sumários
    initial_summary.append(summary)

Dê uma olhada nos primeiros resumos da frase inicial de Map.

In [ ]:
print("\n\n".join(initial_summary[:10]))

Aqui você contará o número de caracteres no resumo inicial para ver se eles são pequenos o suficiente para caber em um prompt.

In [ ]:
len("\n".join(initial_summary))

Como você conseguiu inserir 30.000 caracteres em um prompt anteriormente, também pode inserir todo esse resumo com menos caracteres em um prompt. Você fará isso na próxima etapa.

#### Etapa de Reduce

Aqui você criará uma função de redução que concatenará os resumos da etapa de resumo inicial (etapa Map) e usará o modelo de prompt final para resumir os resumos novamente.

In [ ]:
# define a função que cria o sumário de sumários
def reduce(summary, prompt_template):

    # concatena os sumários da etapa inicial
    concat_summary = "\n".join(initial_summary)

    # cria o prompt para o modelo usando o texto concatenado e a template de prompt
    prompt = prompt_template.format(text=concat_summary)

    # gera o sumário utilizando o modelo e o prompt
    summary = model_with_limit_and_backoff(prompt=prompt, max_output_tokens=1024).text

    return summary

Você está pronto para prosseguir para a próxima etapa para combinar todo o resumo em um resumo ainda menor usando o modelo de prompt final e a função que você criou anteriormente.

In [ ]:
# Use a função `reduce` definida para resumir os resumos
summary = reduce(initial_summary, final_prompt_template)

print(summary)

#### Recapitulando

Você acabou de resumir todo o artigo em alguns marcadores usando o método MapReduce. Aqui estão os prós e contras de usar esse método:

**Prós:**
- Pode resumir um documento grande
- Pode funcionar bem com processamento paralelo, pois os processos para resumir as páginas são independentes entre si

**Contras:**
- Múltiplas chamadas para o modelo são necessárias
- Como as páginas são resumidas individualmente, o contexto entre as páginas pode ser perdido

Na próxima seção, você tentará outro método que usa mais de um bloco (página) por prompt para resumir.

## Método 3: MapReduce com overlapping chunks (ou partes sobrespostos)

É semelhante ao MapReduce, mas com uma diferença fundamental: partes sobrepostas. Isso significa que algumas páginas serão resumidas juntas, em vez de cada página ser resumida separadamente. Isso ajuda a preservar mais contexto ou informações entre os blocos, o que pode melhorar a precisão dos resultados.

É importante observar que a combinação de blocos às vezes pode exceder o limite de token imposto pelo modelo. Se isso ocorrer, você pode implementar o método de divisão de blocos ou resolver o problema de forma criativa (por exemplo, removendo alguns blocos iniciais).

#### Etapa de Map

Nesta seção, você lerá o arquivo PDF novamente e usará o modelo para resumir <b>algumas páginas</b> usando o modelo de prompt inicial definido anteriormente.

In [ ]:
# le o pdf e cria uma lista de páginas
reader = PyPDF2.PdfReader(pdf_file)
pages = reader.pages

# cria uma lista pra armazenar o conteúdo extraído das páginas
text_from_pages = []

# iteração nas páginas pra gerar o sumário de cada página
for page in tqdm(pages):

    # extrai o texto da página e remove os espaços em branco
    text = page.extract_text().strip()

    # append do conteúdo extraído Append the extracted text to the list of extracted text
    text_from_pages.append(text)

Aqui você definirá o tamanho do bloco (número de páginas a serem combinadas neste exemplo) e resumirá os blocos.

In [ ]:
CHUNK_SIZE = 2  # número de páginas combinadas

# le o pdf e cria a lista de páginas
reader = PyPDF2.PdfReader(pdf_file)
pages = reader.pages

# cria a lista que armazenará os sumários
initial_summary = []

# iterage nas páginas e gera o sumário das páginas de acordo com o CHUNK_SIZE
for i in tqdm(range(len(pages))):

    # seleciona a lista de páginas e combina como um único chunk
    pages_to_merge = [x for x in range(i, i + CHUNK_SIZE) if x < len(pages)]

    extracted_texts = [text_from_pages[x] for x in pages_to_merge]

    # concatena o sumário do chunk
    text = "\n".join(extracted_texts)

    # cria o prompt para o modelo utilizando o texto concatenado e a template de prompt
    prompt = initial_prompt_template.format(text=text)

    # gera o sumário usando o modelo e o prompt
    summary = model_with_limit_and_backoff(prompt=prompt, max_output_tokens=1024).text

    # faz append do sumário na lista de sumários
    initial_summary.append(summary)

    # If the last page is reached, break the loop
    if pages_to_merge[-1] == len(reader.pages):
        break

Dê uma olhada nos primeiros resumos da frase inicial do Map.

In [ ]:
print("\n\n".join(initial_summary[:10]))

#### Etapa de Reduce

Você está pronto para prosseguir para a próxima etapa para combinar todo o resumo em um resumo ainda menor usando o modelo de prompt final e a função que você criou anteriormente.

In [ ]:
# usa a função `reduce` para fazer o sumário dos sumários
summary = reduce(initial_summary, final_prompt_template)

print(summary)

#### Recapitulando

O modelo foi capaz de resumir todo o artigo em alguns pontos usando o método MapReduce com Overlapping Chunks. Aqui estão os prós e contras de usar esse método:

**Prós:**
- Pode resumir um documento grande
- Como as páginas sequenciais são resumidas juntas, o contexto entre as páginas é preservado
- Pode usar processamento paralelo, pois os resultados são independentes entre si

**Contras:**
- Múltiplas chamadas para o modelo são necessárias
- Um pouco mais lento que o método MapReduce puro
- Criar texto de entrada maior

Na próxima seção, você tentará uma abordagem diferente que usa um resumo da página anterior em vez do texto inteiro.

## Método 4: MapReduce com resumo contínuo (refinamento)

Em algumas ocasiões, combinar algumas páginas pode ser muito grande para resumir. Para resolver esse problema, agora você terá uma abordagem diferente que usa um resumo inicial da etapa anterior junto com a próxima página para resumir cada prompt. Isso ajuda a garantir que o resumo seja completo e preciso, pois leva em consideração o contexto da página anterior.

In [ ]:
initial_prompt_template = """
    Levando em consideração o contexto abaixo, delimitado por aspas invertidas triplas:

    ```{context}```

    Escreva um sumário conciso sobre o seguinte texto delimitado por aspas invertidas.

    ```{text}```

    SUMÁRIO CONCISO:
"""

In [ ]:
# le o pdf e cria a lista de páginas
reader = PyPDF2.PdfReader(pdf_file)
pages = reader.pages

# cria a lista que armazenará os sumários
initial_summary = []

# # iterage nas páginas e gera o sumário das páginas
for idx, page in enumerate(tqdm(pages)):

    # extrai o texto da página e remove os espaços
    text = page.extract_text().strip()

    if idx == 0:  # se for a primeira página, não há contexto prévio
        prompt = initial_prompt_template.format(context="", text=text)

    else:  # se não for a primeira página, o contexto prévio é o sumário da página anterior
        prompt = initial_prompt_template.format(
            context=initial_summary[idx - 1], text=text
        )

    # gera o sumário utilizando o modelo e a template de prompt
    summary = model_with_limit_and_backoff(prompt=prompt, max_output_tokens=1024).text

    # append do sumário na lista de sumários
    initial_summary.append(summary)

Aqui você listará algumas entradas da lista de resumo inicial.

In [ ]:
print("\n\n".join(initial_summary[:10]))

Espera-se que haja algumas entradas duplicadas na lista, pois você está rolando no contexto das páginas anteriores para a próxima. Você pode facilmente remover essas duplicatas usando a função `set` do Python.

#### Etapa de Reduce
Você está pronto para prosseguir para a próxima etapa para combinar todo o resumo em um resumo ainda menor usando o modelo de prompt final e a função que você criou anteriormente.

In [ ]:
# usa a função `reduce` para fazer o sumário dos sumários
initial_summary = set(initial_summary)  # set() para remover os itens duplicados
summary = reduce(initial_summary, final_prompt_template)

print(summary)

#### Recapitulando

O modelo foi capaz de resumir todo o artigo em alguns pontos usando o método MapReduce with Rolling Summary. Aqui estão os prós e contras de usar esse método:

**Prós:**
- Pode resumir um documento grande
- Como as páginas sequenciais são resumidas usando o contexto das páginas anteriores, o contexto entre as páginas é preservado

**Contras:**
- Múltiplas chamadas para o modelo são necessárias
- Não pode funcionar bem com processamento paralelo, pois os processos para resumir as páginas dependem uns dos outros

## Conclusão

Você resumiu com sucesso um documento longo, mesmo que inicialmente fosse impossível devido a um limite de prompt de entrada. Você também aprendeu vários métodos para resumir documentos longos, junto com suas vantagens e desvantagens.

Resumir um documento longo pode ser um desafio. Exige que você identifique os pontos principais do documento, sintetize as informações e apresente-as de forma concisa e coerente. Isso pode ser especialmente difícil se o documento for complexo ou técnico. Além disso, resumir um documento longo pode ser demorado, pois você precisa ler e analisar cuidadosamente o texto para garantir que o resumo seja preciso e completo.

Embora esses métodos permitam interagir com LLMs e resumir documentos longos de maneira flexível, às vezes você pode querer acelerar o processo usando bootstrapping ou métodos pré-construídos. É aqui que entram as bibliotecas como LangChain. Você pode ler mais sobre o suporte LangChain na Vertex AI [aqui](https://python.langchain.com/en/latest/modules/models/llms/integrations/google_vertex_ai_palm.html).